# 1. IMPORT LIBRARIES AND DATASET

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import string
import re

In [2]:
df = pd.read_excel('../data/vietnamworks.xlsx')
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_name         1344 non-null   object
 1   company          1335 non-null   object
 2   salary           1344 non-null   object
 3   location         1342 non-null   object
 4   skill_1          1336 non-null   object
 5   skill_2          1325 non-null   object
 6   skill_3          1124 non-null   object
 7   job_des          1337 non-null   object
 8   job_requirement  1337 non-null   object
 9   other            1338 non-null   object
 10  Unnamed: 10      1 non-null      object
dtypes: object(11)
memory usage: 115.6+ KB
None
          job_name                  company        salary location  \
count         1344                     1335          1344     1342   
unique        1229                      771           257      128   
top     Kỹ Sư Điện  Navigos Search's Client  Thươn

# PREPROCESS - TÁCH CỘT

## CỘT other
tách các trường từ cột này

In [3]:
print(df.head())

                                            job_name  \
0       Kỹ Sư Thiết Kế Cơ Điện (Upto 20 Triệu/tháng)   
1                   Technical Documentation Engineer   
2  Automation Software Tester (Selenium/java/appium)   
3  Product Manager (Global Market) - Lương Upto $...   
4      Professional Electrical Installation Designer   

                                             company                salary  \
0                       Công Ty Cổ Phần Xây Dựng TEG      Tới 20tr ₫/tháng   
1     CÔNG TY TNHH SPECIALTY BOLT AND SCREW VIỆT NAM          Thương lượng   
2  Bosch Global Software Technologies Company Lim...          Thương lượng   
3                            MISA Jointstock Company  $ 1,000-2,000 /tháng   
4        Danieli – Industrielle Beteiligung Co., Ltd          Thương lượng   

      location                  skill_1                 skill_2  \
0       Hà Nội         Thiết Kế Cơ Điện       Thiết Kế Hệ Thống   
1  Hồ Chí Minh                  Autocad  Mechanical Engineer

### CỘT EXPERIENCE_REQUIRE

In [4]:
def extract_job_info(text):
    if pd.isnull(text):
        return pd.Series([np.nan]*5, index=['level', 'category', 'skills', 'field', 'min_experience'])

    text = text.strip()

    def extract_field(label):
        pattern = rf"{label.upper()}\s*\n(.*?)(?:\n[A-Z]|$)"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
        return np.nan

    level = extract_field("CẤP BẬC")
    category = extract_field("NGÀNH NGHỀ")
    skills = extract_field("KỸ NĂNG")
    field = extract_field("LĨNH VỰC")

    # Xử lý SỐ NĂM KINH NGHIỆM TỐI THIỂU
    exp_raw = extract_field("SỐ NĂM KINH NGHIỆM TỐI THIỂU")
    min_experience = np.nan

    

    if isinstance(exp_raw, str):
        exp_raw_lower = exp_raw.lower()
        if "không yêu cầu" in exp_raw_lower:
            min_experience = 0
        else:
            match = re.search(r'(\d+)', exp_raw_lower)
            if match:
                min_experience = int(match.group(1))
    return pd.Series([level, category, skills, field, min_experience],
                     index=['level', 'category', 'skills', 'field', 'min_experience'])



In [5]:
info_cols = ['level', 'category', 'skills', 'field', 'min_experience']
df[info_cols] = df['other'].apply(extract_job_info)

In [6]:
df.head()

,job_name,company,salary,location,skill_1,skill_2,skill_3,job_des,job_requirement,other,Unnamed: 10,level,category,skills,field,min_experience
0,Kỹ Sư Thiết Kế Cơ Điện (Upto 20 Triệu/tháng),Công Ty Cổ Phần Xây Dựng TEG,Tới 20tr ₫/tháng,Hà Nội,Thiết Kế Cơ Điện,Thiết Kế Hệ Thống,BIM,"Mô tả công việc\n\n- Triển khai tính toán, thi...",Yêu cầu công việc\n\n2. YÊU CẦU ỨNG VIÊN\n\n- ...,Thông tin việc làm\nNGÀY ĐĂNG\n\n15/05/2025\n\...,NaN,Nhân viên,Khoa Học & Kỹ Thuật > Cơ Khí & Điện Lạnh,"Thiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều...",Kỹ thuật xây dựng/Cơ sở hạ tầng,1.0
1,Technical Documentation Engineer,CÔNG TY TNHH SPECIALTY BOLT AND SCREW VIỆT NAM,Thương lượng,Hồ Chí Minh,Autocad,Mechanical Engineering,Quality Management,Mô tả công việc\n\n1. Job Summary\n\nWe are lo...,Yêu cầu công việc\n\nEducation: Bachelor degre...,Thông tin việc làm\nNGÀY ĐĂNG\n\n15/05/2025\n\...,NaN,Mới Tốt Nghiệp,Khoa Học & Kỹ Thuật > Cơ Khí & Điện Lạnh,"Autocad, Mechanical Engineering, Quality Manag...",Ô tô,0.0
2,Automation Software Tester (Selenium/java/appium),Bosch Global Software Technologies Company Lim...,Thương lượng,Hồ Chí Minh,Automation Testing,Java,Appium,"Mô tả công việc\n\n- Develop, maintain and exe...",Yêu cầu công việc\n\n- 1-2 years of experience...,Thông tin việc làm\nNGÀY ĐĂNG\n\n18/04/2025\n\...,NaN,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > QA/QC/Softwar...,"Automation Testing, Java, Appium, Selenium, AP...",Phần Mềm CNTT/Dịch vụ Phần mềm,0.0
3,Product Manager (Global Market) - Lương Upto $...,MISA Jointstock Company,"$ 1,000-2,000 /tháng",Hà Nội,Lập Kế Hoạch Kinh Doanh,Phân Tích Dữ Liệu,Nghiên Cứu Thị Trường,Mô tả công việc\n\nMô tả công việc\n\n• Định h...,Yêu cầu công việc\n\nYêu cầu công việc\n\n• Tố...,Thông tin việc làm\nNGÀY ĐĂNG\n\n21/04/2025\n\...,NaN,Giám Đốc và Cấp Cao Hơn,Công Nghệ Thông Tin/Viễn Thông > Quản Lý Dự Án...,"Lập Kế Hoạch Kinh Doanh, Phân Tích Dữ Liệu, Ng...",Phần Mềm CNTT/Dịch vụ Phần mềm,3.0
4,Professional Electrical Installation Designer,"Danieli – Industrielle Beteiligung Co., Ltd",Thương lượng,Hồ Chí Minh,Electrical Design,ISO Standards,Electrical Installation,Mô tả công việc\n\n• Read and decode block dia...,Yêu cầu công việcQualifications and experience...,Thông tin việc làm\nNGÀY ĐĂNG\n\n21/04/2025\n\...,NaN,Nhân viên,Khoa Học & Kỹ Thuật > Kỹ Thuật Điện/Điện Tử,"Electrical Design, ISO Standards, Electrical I...",Cơ khí/Máy móc/Thiết bị công nghiệp,0.0


In [7]:
# delete all if salary = Thương lượng
df = df[df['salary'] != 'Thương lượng']

In [8]:
df.head()

,job_name,company,salary,location,skill_1,skill_2,skill_3,job_des,job_requirement,other,Unnamed: 10,level,category,skills,field,min_experience
0,Kỹ Sư Thiết Kế Cơ Điện (Upto 20 Triệu/tháng),Công Ty Cổ Phần Xây Dựng TEG,Tới 20tr ₫/tháng,Hà Nội,Thiết Kế Cơ Điện,Thiết Kế Hệ Thống,BIM,"Mô tả công việc\n\n- Triển khai tính toán, thi...",Yêu cầu công việc\n\n2. YÊU CẦU ỨNG VIÊN\n\n- ...,Thông tin việc làm\nNGÀY ĐĂNG\n\n15/05/2025\n\...,NaN,Nhân viên,Khoa Học & Kỹ Thuật > Cơ Khí & Điện Lạnh,"Thiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều...",Kỹ thuật xây dựng/Cơ sở hạ tầng,1.0
3,Product Manager (Global Market) - Lương Upto $...,MISA Jointstock Company,"$ 1,000-2,000 /tháng",Hà Nội,Lập Kế Hoạch Kinh Doanh,Phân Tích Dữ Liệu,Nghiên Cứu Thị Trường,Mô tả công việc\n\nMô tả công việc\n\n• Định h...,Yêu cầu công việc\n\nYêu cầu công việc\n\n• Tố...,Thông tin việc làm\nNGÀY ĐĂNG\n\n21/04/2025\n\...,NaN,Giám Đốc và Cấp Cao Hơn,Công Nghệ Thông Tin/Viễn Thông > Quản Lý Dự Án...,"Lập Kế Hoạch Kinh Doanh, Phân Tích Dữ Liệu, Ng...",Phần Mềm CNTT/Dịch vụ Phần mềm,3.0
5,Facility Officer (Aeon Mall Ha Dong- 459),"AEONMALL Vietnam Co., Ltd.",300-550 ₫/tháng,Hà Nội,Building Management,Facilities Management,Facility Maintenance,"Mô tả công việc\n\nAEONMALL Vietnam Co., Ltd. ...","Yêu cầu công việc\n\n1. Bachelor’s degree, pre...",Thông tin việc làm\nNGÀY ĐĂNG\n\n18/04/2025\n\...,NaN,Nhân viên,Khoa Học & Kỹ Thuật > Điện/Nước/Chất Thải,"Building Management, Facilities Management, Fa...",Bất Động Sản/Cho thuê,1.0
6,C / C++ Software Engineer - Relocation Bonus -...,LG Electronics Development Vietnam Company Lim...,"$ 800-2,500 /tháng",Đà Nẵng,C++,C/C++,Software Development,Mô tả công việc\n\nLG Electronics Development ...,Yêu cầu công việc\n\n1. Requirement:\n\n- Basi...,Thông tin việc làm\nNGÀY ĐĂNG\n\n22/04/2025\n\...,NaN,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Phần Mềm Máy ...,"C++, C/C++, Software Development, OOP",Phần Mềm CNTT/Dịch vụ Phần mềm,1.0
7,Android Devevloper (Relocation Bonus - Hybrid ...,LG Electronics Development Vietnam Company Lim...,"$ 800-2,500 /tháng",Đà Nẵng,Java,Java Spring Framework,Software Engineering,Mô tả công việc\n\nLG Electronics Development ...,Yêu cầu công việc\n\n1. General Requirement:\n...,Thông tin việc làm\nNGÀY ĐĂNG\n\n22/04/2025\n\...,NaN,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Phần Mềm Máy ...,"Java, Java Spring Framework, Software Engineer...",Phần Mềm CNTT/Dịch vụ Phần mềm,0.0


In [9]:
# delete other	Unnamed: 10	
df = df.drop(columns=['other', 'Unnamed: 10'])

In [10]:
experience_counts = df['min_experience'].value_counts()
print("Experience Requirements (from 'min_experience') Counts:")
print(experience_counts)

Experience Requirements (from 'min_experience') Counts:
min_experience
1.0     130
0.0     106
2.0      96
3.0      86
5.0      44
4.0      17
10.0     10
8.0       7
7.0       6
6.0       5
Name: count, dtype: int64


In [11]:
USD_TO_VND = 25_000
VND_TR = 1_000_000

In [12]:

def _clean(num_txt: str) -> float:
    """Loại bỏ dấu , . rồi trả về float."""
    return float(re.sub(r"[,.]", "", num_txt))

def parse_salary(text):
    """Trả về Series(min_salary, max_salary) (đơn vị: VND)."""
    if pd.isnull(text):
        return pd.Series([np.nan, np.nan], index=['min_salary', 'max_salary'])

    txt = str(text).lower().strip()
    txt = txt.replace('–', '-').replace('—', '-')     # chuẩn hóa dấu gạch

    # -------- 1) Khoảng lương a - b --------
    m = re.search(r'(\d[\d,\.]*)\s*(?:tr|triệu)?\s*-\s*(\d[\d,\.]*)', txt)
    if m:
        n1, n2 = map(_clean, m.groups())
        if 'tr' in txt or 'triệu' in txt:           # ưu tiên “tr”
            factor = VND_TR
        elif '$' in txt or 'usd' in txt:
            factor = USD_TO_VND
        else:
            factor = 1                              # giả sử đã là VND đầy đủ
        return pd.Series([n1*factor, n2*factor], index=['min_salary', 'max_salary'])

    # -------- 2) “Từ …”  → min = max --------
    m = re.search(r'\btừ\b\s*\$?\s*(\d[\d,\.]*)\s*(?:tr|triệu)?', txt)
    if m:
        val = _clean(m.group(1))
        if 'tr' in txt or 'triệu' in txt:
            val *= VND_TR
        elif '$' in txt or 'usd' in txt:
            val *= USD_TO_VND
        return pd.Series([val, val], index=['min_salary', 'max_salary'])

    # -------- 3) “Tới/Lên đến …”  → min = max --------
    m = re.search(r'\b(tới|lên đến|đến)\b\s*\$?\s*(\d[\d,\.]*)\s*(?:tr|triệu)?', txt)
    if m:
        val = _clean(m.group(2))
        if 'tr' in txt or 'triệu' in txt:
            val *= VND_TR
        elif '$' in txt or 'usd' in txt:
            val *= USD_TO_VND
        return pd.Series([val, val], index=['min_salary', 'max_salary'])

    # -------- 4) “Trên …”  → min = X, max = NaN --------
    m = re.search(r'\btrên\b\s*\$?\s*(\d[\d,\.]*)\s*(?:tr|triệu)?', txt)
    if m:
        val = _clean(m.group(1))
        if 'tr' in txt or 'triệu' in txt:
            val *= VND_TR
        elif '$' in txt or 'usd' in txt:
            val *= USD_TO_VND
        return pd.Series([val, np.nan], index=['min_salary', 'max_salary'])

    # -------- 5) Một số giá trị đơn lẻ có “tr” --------
    m = re.search(r'\$?\s*(\d[\d,\.]*)\s*(?:tr|triệu)', txt)
    if m:
        val = _clean(m.group(1)) * VND_TR
        return pd.Series([val, val], index=['min_salary', 'max_salary'])

    # -------- 6) Mọi trường hợp còn lại (đơn lẻ, $ …) --------
    m = re.search(r'\$?\s*(\d[\d,\.]*)', txt)
    if m:
        val = _clean(m.group(1))
        if '$' in txt or 'usd' in txt:
            val *= USD_TO_VND
        return pd.Series([val, val], index=['min_salary', 'max_salary'])
    
    return pd.Series([np.nan, np.nan], index=['min_salary', 'max_salary'])

In [13]:
# apply to df
df[['min_salary', 'max_salary']] = df['salary'].apply(parse_salary)

In [14]:
# if min_salary or mmaxsalry = nan, print salary
def check_salary(row):
    if pd.isnull(row['min_salary']) or pd.isnull(row['max_salary']):
        print(f"{row['salary']}")
        return True
    return False

# Check for missing salaries
missing_salaries = df.apply(check_salary, axis=1)


In [15]:
# if in salary find 'năm', devide by 12 for min and max
def adjust_salary_for_yearly(row):
    if pd.isnull(row['min_salary']) or pd.isnull(row['max_salary']):
        return row
    if 'năm' in row['salary'].lower():
        row['min_salary'] /= 12
        row['max_salary'] /= 12
    return row
# Apply the adjustment
df = df.apply(adjust_salary_for_yearly, axis=1)

In [16]:
# if min_salary > 100000000, print salary
def check_high_salary(row):
    if row['min_salary'] > 100_000_000 or row['max_salary'] > 100_000_000:
        print(f"High salary: {row['salary']}")
        return True
    return False
# Check for high salaries
high_salaries = df.apply(check_high_salary, axis=1)

High salary: $ 2,500-5,000 /tháng
High salary: Tới $ 100,000 /năm
High salary: $ 2,000-5,000 /tháng
High salary: $ 20,000-60,000 /năm
High salary: $ 3,000-5,000 /tháng
High salary: $ 25,000-35,000 /tháng
High salary: $ 8,000-12,000 /tháng
High salary: $ 2,500-6,000 /tháng
High salary: $ 2,800-5,000 /tháng
High salary: $ 2,500-30,000 /tháng
High salary: $ 4,000-4,500 /tháng
High salary: $ 6,000-7,000 /tháng
High salary: $ 3,000-5,000 /tháng
High salary: $ 4,000-5,000 /tháng
High salary: $ 4,000-5,000 /tháng
High salary: $ 3,000-5,000 /tháng
High salary: $ 3,000-5,000 /tháng


In [17]:
# if min_salary < 1000000, print salary
def check_low_salary(row):
    if row['min_salary'] < 1_000_000 or row['max_salary'] < 1_000_000:
        print(f"Low salary: {row['salary']}")
        return True
    return False
# Check for low salaries
low_salaries = df.apply(check_low_salary, axis=1)
# delete all rows
df = df[~low_salaries]

Low salary: 300-550 ₫/tháng
Low salary: 10,000-20,000 ₫/tháng
Low salary: 500-800 ₫/tháng
Low salary: Tới 2,000 ₫/tháng
Low salary: 450-550 ₫/tháng
Low salary: 700-1,500 ₫/tháng
Low salary: 650-1,000 ₫/tháng
Low salary: 400-600 ₫/tháng
Low salary: 700-1,500 ₫/tháng
Low salary: 1,000-1,500 ₫/tháng
Low salary: 900-1,200 ₫/tháng
Low salary: 900-1,300 ₫/tháng
Low salary: ¥ 300,000-700,000 /tháng
Low salary: Tới 500 ₫/tháng
Low salary: Tới 1,200 ₫/tháng
Low salary: 30-40 ₫/tháng


In [18]:
df.head()

,job_name,company,salary,location,skill_1,skill_2,skill_3,job_des,job_requirement,level,category,skills,field,min_experience,min_salary,max_salary
0,Kỹ Sư Thiết Kế Cơ Điện (Upto 20 Triệu/tháng),Công Ty Cổ Phần Xây Dựng TEG,Tới 20tr ₫/tháng,Hà Nội,Thiết Kế Cơ Điện,Thiết Kế Hệ Thống,BIM,"Mô tả công việc\n\n- Triển khai tính toán, thi...",Yêu cầu công việc\n\n2. YÊU CẦU ỨNG VIÊN\n\n- ...,Nhân viên,Khoa Học & Kỹ Thuật > Cơ Khí & Điện Lạnh,"Thiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều...",Kỹ thuật xây dựng/Cơ sở hạ tầng,1.0,20000000.0,20000000.0
3,Product Manager (Global Market) - Lương Upto $...,MISA Jointstock Company,"$ 1,000-2,000 /tháng",Hà Nội,Lập Kế Hoạch Kinh Doanh,Phân Tích Dữ Liệu,Nghiên Cứu Thị Trường,Mô tả công việc\n\nMô tả công việc\n\n• Định h...,Yêu cầu công việc\n\nYêu cầu công việc\n\n• Tố...,Giám Đốc và Cấp Cao Hơn,Công Nghệ Thông Tin/Viễn Thông > Quản Lý Dự Án...,"Lập Kế Hoạch Kinh Doanh, Phân Tích Dữ Liệu, Ng...",Phần Mềm CNTT/Dịch vụ Phần mềm,3.0,25000000.0,50000000.0
6,C / C++ Software Engineer - Relocation Bonus -...,LG Electronics Development Vietnam Company Lim...,"$ 800-2,500 /tháng",Đà Nẵng,C++,C/C++,Software Development,Mô tả công việc\n\nLG Electronics Development ...,Yêu cầu công việc\n\n1. Requirement:\n\n- Basi...,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Phần Mềm Máy ...,"C++, C/C++, Software Development, OOP",Phần Mềm CNTT/Dịch vụ Phần mềm,1.0,20000000.0,62500000.0
7,Android Devevloper (Relocation Bonus - Hybrid ...,LG Electronics Development Vietnam Company Lim...,"$ 800-2,500 /tháng",Đà Nẵng,Java,Java Spring Framework,Software Engineering,Mô tả công việc\n\nLG Electronics Development ...,Yêu cầu công việc\n\n1. General Requirement:\n...,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Phần Mềm Máy ...,"Java, Java Spring Framework, Software Engineer...",Phần Mềm CNTT/Dịch vụ Phần mềm,0.0,20000000.0,62500000.0
14,Data Engineer,Techcombank,"$ 1,000-2,500 /tháng",Hà Nội,Banking,Data Engineer,Computer Science,Mô tả công việc\n\n• Lead the team technically...,Yêu cầu công việc\n\n1. Must have:\n\n• Bachel...,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Data Engineer...,"Banking, Data Engineer, Computer Science, Info...",Ngân hàng,3.0,25000000.0,62500000.0


In [19]:
# thêm 2 cột tên exp_2, education, toàn bộ đều là NaN
df['exp_2'] = np.nan
df['education'] = np.nan

In [22]:
def normalize_text(text):
    text = text.lower()
    text = text.replace("’", "'").replace("‘", "'").replace("“", '"').replace("”", '"')
    return text

def parse_job_require(text):
    if pd.isnull(text):
        return pd.Series([np.nan, np.nan], index=['degree', 'experience'])

    txt = normalize_text(str(text))

    # -------- 1. Degree detection --------
    degrees = []
    degree = np.nan  # ✅ Khởi tạo để phòng lỗi NameError

    degree_patterns = {
        'phd': r'(tiến sĩ|doctorate|ph\.?\s*d)',
        'master': r"(thạc sĩ|master'?s?\s+degree|msc)",
        'bachelor': r"(đại\s*học|đh|bachelor'?s?\s+degree|bachelor\s+degree|graduated\s+from\s+university|university\s+degree)",
        'college': r"(cao\s*đẳng|college\s+degree)",
        'associate': r"(trung\s*cấp|associate\s+degree|trung\s*cấp\s*nghề)",
        'highschool': r"(thpt|trung học phổ thông|high\s*school)",
        'secondary': r"(thcs|trung học cơ sở|secondary\s*school)"
    }

    for key, pattern in degree_patterns.items():
        if re.search(pattern, txt, flags=re.IGNORECASE):
            degrees.append(key)

    # ➕ Bắt bằng keyword nếu chưa tìm được degree rõ ràng
    if not degrees:
        soft_bachelor_keywords = [
            r"sinh\s*viên",
            r"năm\s*cuối",
            r"final\s*year",
            r"tốt\s*nghiệp\s+đại\s*học",
            r"graduate(d)?\s+from\s+university",
            r"graduated\s+university",
            r"graduated",
            r"tốt\s*nghiệp",
            r"cử\s*nhân"
        ]
        if any(re.search(pat, txt, flags=re.IGNORECASE) for pat in soft_bachelor_keywords):
            degrees.append('bachelor')

    if degrees:
        degree = ', '.join(sorted(set(degrees)))

    # -------- 2. Experience detection --------
    experience = np.nan
    matches = re.findall(
        r'(?:at least|min(?:imum)? of|ít nhất|tối thiểu|khoảng|~)?\s*(\d+)(?:\s*[-–]\s*(\d+)|\+)?\s*(?:năm|years?)',
        txt, flags=re.IGNORECASE
    )

    if matches:
        numbers = []
        for a, b in matches:
            numbers.append(int(a))
            if b:
                numbers.append(int(b))
        experience = round(np.mean(numbers), 1)

    return pd.Series([degree, experience], index=['degree', 'experience'])


In [23]:
df[['education', 'exp_2']] = df['job_requirement'].apply(parse_job_require)


In [24]:
# print require if education is NaN
def check_exp_2(row):
    if pd.isnull(row['exp_2']):
        print(f"Missing exp_2 for job: {row['job_requirement']}")
        return True
    return False
# Check for missing exp_2
missing_exp_2 = df.apply(check_exp_2, axis=1)


Missing exp_2 for job: Yêu cầu công việc

Yêu cầu công việc

• Tốt nghiệp đại học chuyên ngành quản trị kinh doanh, kinh tế từ các trường ĐH Ngoại thương, ĐH Kinh tế Quốc dân, hoặc các trường tương đương.

• Có khả năng nghiên cứu thị trường và khách hàng một cách hệ thống; có kỹ năng phân tích dữ liệu để đưa ra quyết định; khả năng lập kế hoạch kinh doanh và phát triển sản phẩm hiệu quả.

• Kỹ năng giao tiếp, trình bày và thuyết phục tốt.

• Khả năng làm việc nhóm và giải quyết vấn đề hiệu quả.

• Trình độ tiếng Anh: Nghe, Nói, Đọc, Viết từ mức Khá trở lên; có khả năng giao tiếp thành thạo bằng tiếng Anh với đối tác và khách hàng quốc tế.

• Đam mê và có định hướng lâu dài trong lĩnh vực phát triển sản phẩm công nghệ; quan tâm đến việc ứng dụng công nghệ để nâng cao hiệu quả công việc; mong muốn tạo ra sản phẩm mang lại giá trị cho xã hội.

• Ưu tiên ứng viên có kinh nghiệm làm việc ở vị trí Senior BA/PO/PM trong các công ty phần mềm có sản phẩm/dịch vụ liên quan đến lĩnh vực F&B.

• 

In [25]:
# in job_des and job_requirement, if '\n' in text, replace it with ' ', if multiple ' ' replace by one ' '
def normalize_text(text):
    if pd.isnull(text):
        return text
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)  # replace multiple spaces with one space
    return text.strip()

# in job_des and job_requirement, if '\n' in text, replace it with ' ', if multiple ' ' replace by one ' '
df['job_des'] = df['job_des'].apply(normalize_text)
df['job_requirement'] = df['job_requirement'].apply(normalize_text)

In [26]:
# delete salary skill_1	skill_2	skill_3	
df = df.drop(columns=['salary', 'skill_1', 'skill_2', 'skill_3'])

In [28]:
# delete if min_experience = nan
df = df.dropna(subset=['min_experience'])

In [30]:
df.head()
# if exp_2 = nan -> continue, else, if exp_2 > min_experience, min exp = exp_2
def adjust_min_experience(row):
    if pd.isnull(row['exp_2']):
        return row
    if row['exp_2'] > row['min_experience']:
        row['min_experience'] = row['exp_2']
    return row

df = df.apply(adjust_min_experience, axis=1)

In [31]:
df.head()

,job_name,company,location,job_des,job_requirement,level,category,skills,field,min_experience,min_salary,max_salary,exp_2,education
0,Kỹ Sư Thiết Kế Cơ Điện (Upto 20 Triệu/tháng),Công Ty Cổ Phần Xây Dựng TEG,Hà Nội,"Mô tả công việc - Triển khai tính toán, thiết ...",Yêu cầu công việc 2. YÊU CẦU ỨNG VIÊN - Tốt ng...,Nhân viên,Khoa Học & Kỹ Thuật > Cơ Khí & Điện Lạnh,"Thiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều...",Kỹ thuật xây dựng/Cơ sở hạ tầng,1.0,20000000.0,20000000.0,1.0,bachelor
3,Product Manager (Global Market) - Lương Upto $...,MISA Jointstock Company,Hà Nội,Mô tả công việc Mô tả công việc • Định hướng v...,Yêu cầu công việc Yêu cầu công việc • Tốt nghi...,Giám Đốc và Cấp Cao Hơn,Công Nghệ Thông Tin/Viễn Thông > Quản Lý Dự Án...,"Lập Kế Hoạch Kinh Doanh, Phân Tích Dữ Liệu, Ng...",Phần Mềm CNTT/Dịch vụ Phần mềm,3.0,25000000.0,50000000.0,NaN,bachelor
6,C / C++ Software Engineer - Relocation Bonus -...,LG Electronics Development Vietnam Company Lim...,Đà Nẵng,Mô tả công việc LG Electronics Development Vie...,Yêu cầu công việc 1. Requirement: - Basic Qual...,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Phần Mềm Máy ...,"C++, C/C++, Software Development, OOP",Phần Mềm CNTT/Dịch vụ Phần mềm,1.0,20000000.0,62500000.0,1.0,bachelor
7,Android Devevloper (Relocation Bonus - Hybrid ...,LG Electronics Development Vietnam Company Lim...,Đà Nẵng,Mô tả công việc LG Electronics Development Vie...,Yêu cầu công việc 1. General Requirement: _ Ba...,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Phần Mềm Máy ...,"Java, Java Spring Framework, Software Engineer...",Phần Mềm CNTT/Dịch vụ Phần mềm,2.0,20000000.0,62500000.0,2.0,bachelor
14,Data Engineer,Techcombank,Hà Nội,Mô tả công việc • Lead the team technically in...,Yêu cầu công việc 1. Must have: • Bachelor’s o...,Nhân viên,Công Nghệ Thông Tin/Viễn Thông > Data Engineer...,"Banking, Data Engineer, Computer Science, Info...",Ngân hàng,3.0,25000000.0,62500000.0,3.0,master


In [32]:
# drop exp_2
df = df.drop(columns=['exp_2'])

In [33]:
# if education = nan, replace it with 'unknown'
df['education'] = df['education'].fillna('unknown')

In [34]:
# save to xlsx
df.to_excel('../data/vietnamworks_preprocessed.xlsx', index=False)

## COMPANY NAME


In [17]:
# Xóa dấu chấm cuối câu, xóa khoảng trống thừa, chuyển về chữ thường
df['company'] = df['company'].apply(lambda x: x.strip().lower().capitalize() if isinstance(x, str) else x)

# viết hoa đầu mỗi từ
df['company'] = df['company'].apply(lambda x: string.capwords(x) if isinstance(x, str) else x)

# print unique
print(df['company'].nunique())

758


In [18]:
print(df['company'].value_counts().head(10))

company
Navigos Search's Client                               42
Ngân Hàng Thương Mại Cổ Phần Quân Đội - Mb            28
Navigos Search                                        22
Techcombank                                           17
Bosch Global Software Technologies Company Limited    16
Công Ty Cổ Phần Tập Đoàn Golden Gate                  10
Công Ty Cổ Phần Chứng Khoán Ssi                        9
Vinfast Trading And Production Jsc                     8
Heineken Vietnam                                       8
Công Ty Cổ Phần Thanh Toán Quốc Gia Việt Nam           8
Name: count, dtype: int64


## CỘT pther

In [21]:
# Hàm xử lý từng dòng trong cột 'general_info'
def process_other(text):
    # Loại bỏ '\n' và khoảng trắng thừa
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Lấy đoạn giữa "Cấp bậc" và "Số lượng tuyển"
    # Nếu không tìm thấy thì trả về chuỗi rỗng
    position = text[text.find('Cấp bậc') + len('Cấp bậc:'):text.find('Số lượng tuyển')].strip()

    # Lấy đoạn giữa "Hình thức làm việc" và "Giới tính"
    # Nếu không tìm Giới tính thì in ra chuỗi sau "Hình thức làm việc"
    work_type = text[text.find('Hình thức làm việc') + len('Hình thức làm việc:'):text.find('Giới tính')].strip()

    return position, work_type

In [22]:
text = df['other'].iloc[0]
print(process_other(text))

('in việc làmNGÀY ĐĂNG15/05/2025CẤP BẬCNhân viênNGÀNH NGHỀKhoa Học & Kỹ Thuật > Cơ Khí & Điện LạnhKỸ NĂNGThiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều Hòa Thông Gió, Cơ ĐiệnLĨNH VỰCKỹ thuật xây dựng/Cơ sở hạ tầngNGÔN NGỮ TRÌNH BÀY HỒ SƠBất kỳSỐ NĂM KINH NGHIỆM TỐI THIỂU1QUỐC TỊCHKhông hiển thịXem thê', 'NGÀY ĐĂNG15/05/2025CẤP BẬCNhân viênNGÀNH NGHỀKhoa Học & Kỹ Thuật > Cơ Khí & Điện LạnhKỸ NĂNGThiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều Hòa Thông Gió, Cơ ĐiệnLĨNH VỰCKỹ thuật xây dựng/Cơ sở hạ tầngNGÔN NGỮ TRÌNH BÀY HỒ SƠBất kỳSỐ NĂM KINH NGHIỆM TỐI THIỂU1QUỐC TỊCHKhông hiển thịXem thê')


In [24]:
# Fill NaN values with an empty string
df['other'] = df['other'].fillna('')

# Áp dụng hàm cho toàn bộ cột 'other'
df['position'], df['work_type'] = zip(*df['other'].apply(process_other))

# Kết quả
print(df)

                                               job_name  \
0          Kỹ Sư Thiết Kế Cơ Điện (Upto 20 Triệu/tháng)   
1                      Technical Documentation Engineer   
2     Automation Software Tester (Selenium/java/appium)   
3     Product Manager (Global Market) - Lương Upto $...   
4         Professional Electrical Installation Designer   
...                                                 ...   
1339       Chuyên Viên IT Helpdesk _Ưu Tiên Đi Lam Ngay   
1340            Technical Lead (Nextjs & Nestjs) Remote   
1341            Technical Lead (Nextjs & Nestjs)_Remote   
1342    NHÂN VIÊN KỸ THUẬT ĐIỆN (Electrical Technician)   
1343                Nghiên Cứu Ứng Dụng AI & Automation   

                                                company                salary  \
0                          Công Ty Cổ Phần Xây Dựng Teg      Tới 20tr ₫/tháng   
1        Công Ty Tnhh Specialty Bolt And Screw Việt Nam          Thương lượng   
2     Bosch Global Software Technologies Company

In [25]:
#delete all in position after "Học vấn"
df['position'] = df['position'].apply(lambda x: x.split('Học vấn')[0])
# delete blank
df['position'] = df['position'].apply(lambda x: x.strip())

In [26]:
#if position = '', then fill with "Nhân viên"
df['position'] = df['position'].apply(lambda x: 'Nhân viên' if x == '' else x)

In [27]:
#print unique
print(df['position'].unique())

['in việc làmNGÀY ĐĂNG15/05/2025CẤP BẬCNhân viênNGÀNH NGHỀKhoa Học & Kỹ Thuật > Cơ Khí & Điện LạnhKỸ NĂNGThiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều Hòa Thông Gió, Cơ ĐiệnLĨNH VỰCKỹ thuật xây dựng/Cơ sở hạ tầngNGÔN NGỮ TRÌNH BÀY HỒ SƠBất kỳSỐ NĂM KINH NGHIỆM TỐI THIỂU1QUỐC TỊCHKhông hiển thịXem thê'
 'in việc làmNGÀY ĐĂNG15/05/2025CẤP BẬCMới Tốt NghiệpNGÀNH NGHỀKhoa Học & Kỹ Thuật > Cơ Khí & Điện LạnhKỸ NĂNGAutocad, Mechanical Engineering, Quality Management, SolidWorks, MechanicalLĨNH VỰCÔ tôNGÔN NGỮ TRÌNH BÀY HỒ SƠTiếng AnhSỐ NĂM KINH NGHIỆM TỐI THIỂUKhông yêu cầuQUỐC TỊCHKhông hiển thịXem thê'
 'in việc làmNGÀY ĐĂNG18/04/2025CẤP BẬCNhân viênNGÀNH NGHỀCông Nghệ Thông Tin/Viễn Thông > QA/QC/Software TestingKỸ NĂNGAutomation Testing, Java, Appium, Selenium, API TestingLĨNH VỰCPhần Mềm CNTT/Dịch vụ Phần mềmNGÔN NGỮ TRÌNH BÀY HỒ SƠBất kỳSỐ NĂM KINH NGHIỆM TỐI THIỂUKhông yêu cầuQUỐC TỊCHKhông hiển thịXem thê'
 ...
 'in việc làmNGÀY ĐĂNG12/04/2025CẤP BẬCTrưởng phòngNGÀNH NGHỀCông Nghệ T

In [28]:
#delete all after "Cấp bậc"
df['work_type'] = df['work_type'].apply(lambda x: x.split('Cấp bậc')[0])
df['work_type'] = df['work_type'].apply(lambda x: x.split('Yêu cầu khác')[0])
# delete blank
df['work_type'] = df['work_type'].apply(lambda x: x.strip())
#toàn thời gia -> toàn thời gian
# toàn thời gian -> giữ nguyên
df['work_type'] = df['work_type'].apply(lambda x: 'Toàn thời gian' if x == 'Toàn thời gia' else x)
df['work_type'] = df['work_type'].apply(lambda x: 'Bán thời gian' if x == 'Bán thời gia' else x)
df['work_type'] = df['work_type'].apply(lambda x: 'Thực tập' if x == 'Thực tậ' else x)

In [29]:
print(df['work_type'].unique())

['NGÀY ĐĂNG15/05/2025CẤP BẬCNhân viênNGÀNH NGHỀKhoa Học & Kỹ Thuật > Cơ Khí & Điện LạnhKỸ NĂNGThiết Kế Cơ Điện, Thiết Kế Hệ Thống, BIM, Điều Hòa Thông Gió, Cơ ĐiệnLĨNH VỰCKỹ thuật xây dựng/Cơ sở hạ tầngNGÔN NGỮ TRÌNH BÀY HỒ SƠBất kỳSỐ NĂM KINH NGHIỆM TỐI THIỂU1QUỐC TỊCHKhông hiển thịXem thê'
 'NGÀY ĐĂNG15/05/2025CẤP BẬCMới Tốt NghiệpNGÀNH NGHỀKhoa Học & Kỹ Thuật > Cơ Khí & Điện LạnhKỸ NĂNGAutocad, Mechanical Engineering, Quality Management, SolidWorks, MechanicalLĨNH VỰCÔ tôNGÔN NGỮ TRÌNH BÀY HỒ SƠTiếng AnhSỐ NĂM KINH NGHIỆM TỐI THIỂUKhông yêu cầuQUỐC TỊCHKhông hiển thịXem thê'
 'NGÀY ĐĂNG18/04/2025CẤP BẬCNhân viênNGÀNH NGHỀCông Nghệ Thông Tin/Viễn Thông > QA/QC/Software TestingKỸ NĂNGAutomation Testing, Java, Appium, Selenium, API TestingLĨNH VỰCPhần Mềm CNTT/Dịch vụ Phần mềmNGÔN NGỮ TRÌNH BÀY HỒ SƠBất kỳSỐ NĂM KINH NGHIỆM TỐI THIỂUKhông yêu cầuQUỐC TỊCHKhông hiển thịXem thê'
 ...
 'NGÀY ĐĂNG12/04/2025CẤP BẬCTrưởng phòngNGÀNH NGHỀCông Nghệ Thông Tin/Viễn Thông > Phần Mềm Máy TínhKỸ NĂ

In [ ]:
df['other_info'].head()

4    Thông tin chung\n    \n        \n    \n       ...
5    Thông tin chung\n    \n        \n    \n       ...
6    Thông tin chung\n    \n        \n    \n       ...
7    Thông tin chung\n    \n        \n    \n       ...
8    Thông tin chung\n    \n        \n    \n       ...
Name: other_info, dtype: object

In [30]:
df['education'] = df['other_info']

# if \n -> ' ' -> ' ', then strip, lấy đoạn giữa "Học vấn" và "số lượng tuyển"
# if dont find "Học vấn" in other_info, return nan
def extract_education(text):
    if pd.isna(text):
        return np.nan
    text = re.sub(r'\n', ' ', text)  # replace newlines with space
    text = re.sub(r'\s+', ' ', text)  # replace multiple spaces with single space
    text = text.strip()  # remove leading and trailing spaces
    match = re.search(r'Học vấn\s*(.*?)\s*Số lượng tuyển', text)
    if match:
        return match.group(1).strip()
    else:
        return np.nan  # return NaN if "Học vấn" not found
    
# apply the function to education column
df['education'] = df['education'].apply(extract_education)


KeyError: 'other_info'

In [ ]:
df['education'].unique()

array(['Cao Đẳng trở lên', 'Đại Học trở lên', 'Trung cấp trở lên',
       'Trung học phổ thông (Cấp 3) trở lên', 'Cao học trở lên',
       'Trung học cơ sở (Cấp 2) trở lên'], dtype=object)

## CỘT LOCATION

In [ ]:
provinces = [
    "Hà Nội",
    "Vĩnh Phúc",
    "Bắc Ninh",
    "Quảng Ninh",
    "Hải Dương",
    "Hải Phòng",
    "Hưng Yên",
    "Thái Bình",
    "Hà Nam",
    "Nam Định",
    "Ninh Bình",
    "Hà Giang",
    "Cao Bằng",
    "Bắc Kạn",
    "Tuyên Quang",
    "Lào Cai",
    "Yên Bái",
    "Thái Nguyên",
    "Lạng Sơn",
    "Bắc Giang",
    "Phú Thọ",
    "Điện Biên",
    "Lai Châu",
    "Sơn La",
    "Hoà Bình",
    "Thanh Hoá",
    "Nghệ An",
    "Hà Tĩnh",
    "Quảng Bình",
    "Quảng Trị",
    "Thừa Thiên Huế",
    "Đà Nẵng",
    "Quảng Nam",
    "Quảng Ngãi",
    "Bình Định",
    "Phú Yên",
    "Khánh Hoà",
    "Ninh Thuận",
    "Bình Thuận",
    "Kon Tum",
    "Gia Lai",
    "Đắk Lắk",
    "Đắk Nông",
    "Lâm Đồng",
    "Bình Phước",
    "Tây Ninh",
    "Bình Dương",
    "Đồng Nai",
    "Bà Rịa - Vũng Tàu",
    "Hồ Chí Minh",
    "Long An",
    "Tiền Giang",
    "Bến Tre",
    "Trà Vinh",
    "Vĩnh Long",
    "Đồng Tháp",
    "An Giang",
    "Kiên Giang",
    "Cần Thơ",
    "Hậu Giang",
    "Sóc Trăng",
    "Bạc Liêu",
    "Cà Mau"
]

# In ra danh sách
for province in provinces:
    print(province)

Hà Nội
Vĩnh Phúc
Bắc Ninh
Quảng Ninh
Hải Dương
Hải Phòng
Hưng Yên
Thái Bình
Hà Nam
Nam Định
Ninh Bình
Hà Giang
Cao Bằng
Bắc Kạn
Tuyên Quang
Lào Cai
Yên Bái
Thái Nguyên
Lạng Sơn
Bắc Giang
Phú Thọ
Điện Biên
Lai Châu
Sơn La
Hoà Bình
Thanh Hoá
Nghệ An
Hà Tĩnh
Quảng Bình
Quảng Trị
Thừa Thiên Huế
Đà Nẵng
Quảng Nam
Quảng Ngãi
Bình Định
Phú Yên
Khánh Hoà
Ninh Thuận
Bình Thuận
Kon Tum
Gia Lai
Đắk Lắk
Đắk Nông
Lâm Đồng
Bình Phước
Tây Ninh
Bình Dương
Đồng Nai
Bà Rịa - Vũng Tàu
Hồ Chí Minh
Long An
Tiền Giang
Bến Tre
Trà Vinh
Vĩnh Long
Đồng Tháp
An Giang
Kiên Giang
Cần Thơ
Hậu Giang
Sóc Trăng
Bạc Liêu
Cà Mau


In [ ]:
def process_location(text):
    # Loại bỏ '\n' và khoảng trắng thừa
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [ ]:
# Fill NaN values with an empty string
df['city_text'] = df['city_text'].fillna('')

# áp dụng vào cột Location
df['city_text'] = df['city_text'].apply(process_location)

# Kết quả
print(df['city_text'])


4            Hà Nội
5            Hà Nội
6       Hồ Chí Minh
7       Hồ Chí Minh
8       Hồ Chí Minh
           ...     
2364         Hà Nội
2365         Hà Nội
2366         Hà Nội
2367         Hà Nội
2368         Hà Nội
Name: city_text, Length: 2310, dtype: object


In [ ]:
# Hàm tìm các tỉnh thành trong văn bản, các hàng có nhiều hơn 1 tỉnh, ngăn cách nhau bởi dấu ','
def find_provinces(text):
    # Tìm các tỉnh thành trong văn bản
    found_provinces = [province for province in provinces if province in text]
    
    # Nếu không tìm thấy thì trả về chuỗi rỗng
    if len(found_provinces) == 0:
        return ''
    
    # Nếu tìm thấy thì trả về chuỗi các tỉnh thành, ngăn cách nhau bởi dấu ','
    return ', '.join(found_provinces)

# Áp dụng hàm cho toàn bộ cột 'Location'
df['city_text'] = df['city_text'].apply(find_provinces)

# Kết quả
print(df['city_text'])

4            Hà Nội
5            Hà Nội
6       Hồ Chí Minh
7       Hồ Chí Minh
8       Hồ Chí Minh
           ...     
2364         Hà Nội
2365         Hà Nội
2366         Hà Nội
2367         Hà Nội
2368         Hà Nội
Name: city_text, Length: 2310, dtype: object


In [ ]:
# print unique
print(df['city_text'].unique())

['Hà Nội' 'Hồ Chí Minh' 'Hà Nội, Hồ Chí Minh' 'Thái Nguyên' 'Nghệ An'
 'Hải Phòng' 'Long An' 'Đồng Nai' 'Đà Nẵng' 'Hà Nội, Hưng Yên' 'Khánh Hoà'
 'Gia Lai' 'Hưng Yên' 'Thanh Hoá' 'Hải Dương' 'Bình Dương'
 'Hà Nội, Nghệ An' 'Thừa Thiên Huế' 'Hồ Chí Minh, Long An'
 'Hà Nội, Vĩnh Phúc' 'Vĩnh Phúc' 'Bình Định' 'Hà Nội, Đà Nẵng'
 'Quảng Ninh, Thanh Hoá' 'Hà Nội, Hải Phòng' 'Hà Nam' 'Bắc Ninh'
 'Đà Nẵng, Hồ Chí Minh' 'Lâm Đồng' 'Bắc Giang' 'Ninh Bình' 'Cần Thơ'
 'Nghệ An, Đà Nẵng' 'Phú Yên' '' 'Đồng Tháp' 'Bình Dương, Hồ Chí Minh']


In [ ]:
# count print if null
print("Number of rows with null city_text:", df['city_text'].isnull().sum())

Number of rows with null city_text: 0


# PREPROCESS - XỬ LÝ CỘT DỮ LIỆU ĐẶC BIỆT
- Lương
- Kinh nghiệm


## Salary

In [ ]:
# print unique salary
print(df['title_salary'].unique())


['10 - 18 triệu' '10 - 20 triệu' '12 - 15 triệu' '12 - 18 triệu'
 '13 - 23 triệu' '15 - 20 triệu' '18 - 22 triệu' '20 - 27 triệu'
 '20 - 40 triệu' '20 - 50 triệu' '25 - 35 triệu' '25 - 50 triệu'
 '35 - 40 triệu' '4 - 6 triệu' '5 - 6 triệu' '8 - 12 triệu' '8 - 15 triệu'
 '800 - 3,500 USD' 'Tới 1,500 USD' 'Tới 2 triệu' 'Tới 20 triệu'
 'Tới 3 triệu' 'Tới 6,000 USD' 'Từ 1,200 USD' '20 - 35 triệu'
 'Tới 25 triệu' 'Tới 35 triệu' '8 - 16 triệu' 'Tới 26 triệu'
 '15 - 18 triệu' '28 - 33 triệu' '30 - 45 triệu' 'Tới 40 triệu'
 '12 - 25 triệu' '35 - 55 triệu' '2,000 - 3,000 USD' '12 - 19 triệu'
 '15 - 25 triệu' '18 - 25 triệu' 'Tới 30 triệu' 'Tới 60 triệu'
 '13 - 15 triệu' 'Tới 1,000 USD' 'Tới 700 USD' '12 - 35 triệu'
 '8 - 30 triệu' '9 - 15 triệu' 'Tới 50 triệu' '23 - 32 triệu'
 '6.9 - 7.3 triệu' 'Từ 16 triệu' 'Từ 50 triệu' '14 - 17 triệu'
 '12 - 20 triệu' '16 - 18 triệu' '9 - 11 triệu' '17 - 50 triệu'
 '30 - 46 triệu' '8 - 10 triệu' '3 - 5 triệu' '9 - 14 triệu'
 '15 - 50 triệu' '25 - 40 triệu' '

In [ ]:
# in title_salary, delete all after  Số lượng tuyển 
df['title_salary'] = df['title_salary'].apply(lambda x: x.split('Số lượng tuyển')[0] if 'Số lượng tuyển' in x else x)
# in title_salary, delete all before Thu nhập
df['title_salary'] = df['title_salary'].apply(lambda x: x.split('Thu nhập')[1] if 'Thu nhập' in x else x)
# delete blank
df['title_salary'] = df['title_salary'].apply(lambda x: x.strip())

In [ ]:
# Hàm xử lý dữ liệu lương
def process_salary(text):
    # Loại bỏ khoảng trắng thừa
    text = text.strip()
    min_salary = max_salary = None

    # Nếu tìm thấy 'USD' thì nhân với 25485
    if 'USD' in text:
        salary_text = text.replace('USD', '').replace('usd', '').strip()
        # Xử lý trường hợp khoảng hoặc dấu gạch ngang
        if '-' in salary_text:
            parts = salary_text.split('-')
        elif '–' in salary_text:
            parts = salary_text.split('–')
        else:
            parts = [salary_text]
        parts = [p.replace('.', '').replace(',', '').strip() for p in parts]
        try:
            if len(parts) == 2:
                min_salary = int(float(parts[0])) * 25485
                max_salary = int(float(parts[1])) * 25485
            else:
                min_salary = max_salary = int(float(parts[0])) * 25485
        except Exception:
            min_salary = max_salary = None
    else:
        # Xử lý trường hợp triệu, tr, hoặc chỉ số
        salary_text = text.replace('triệu', '').replace('tr', '').strip()
        if '-' in salary_text:
            parts = salary_text.split('-')
        elif '–' in salary_text:
            parts = salary_text.split('–')
        else:
            parts = [salary_text]
        parts = [p.replace('.', '').replace(',', '').strip() for p in parts]
        try:
            if len(parts) == 2:
                min_salary = int(float(parts[0])) * 1000000
                max_salary = int(float(parts[1])) * 1000000
            else:
                min_salary = max_salary = int(float(parts[0])) * 1000000
        except Exception:
            min_salary = max_salary = None
    return min_salary, max_salary

# Áp dụng hàm cho toàn bộ cột 'title_salary'
df['min_salary'], df['max_salary'] = zip(*df['title_salary'].apply(process_salary))

# In kết quả
print(df[['min_salary', 'max_salary']])


      min_salary  max_salary
4     10000000.0  18000000.0
5     10000000.0  18000000.0
6     10000000.0  18000000.0
7     10000000.0  18000000.0
8     10000000.0  20000000.0
...          ...         ...
2364  18000000.0  35000000.0
2365  18000000.0  35000000.0
2366  22000000.0  30000000.0
2367   8000000.0  15000000.0
2368   8000000.0  16000000.0

[2310 rows x 2 columns]


In [ ]:
# in type của min_salary và max_salary
print(df['min_salary'].dtype)

float64


In [ ]:
# if max_salary > 100000000, print min_salary, max_salary, title_salary
df.loc[df['max_salary'] > 100000000, ['min_salary', 'max_salary', 'title_salary']]

,min_salary,max_salary,title_salary
265,22000000.0,255000000.0,22 - 25.5 triệu
757,25485000.0,101940000.0,"1,000 - 4,000 USD"
1344,50970000.0,101940000.0,"2,000 - 4,000 USD"
1351,89197500.0,101940000.0,"3,500 - 4,000 USD"
1438,50970000.0,101940000.0,"2,000 - 4,000 USD"
1439,50970000.0,101940000.0,"2,000 - 4,000 USD"
1514,63712500.0,101940000.0,"2,500 - 4,000 USD"
1585,15000000.0,220000000.0,15 - 22.0 triệu
1656,63712500.0,101940000.0,"2,500 - 4,000 USD"
1787,76455000.0,101940000.0,"3,000 - 4,000 USD"


In [ ]:
# if title_salary have '.', print min_salary, max_salary
df.loc[df['title_salary'].str.contains(r'\.'), ['min_salary', 'max_salary', 'title_salary']]

,min_salary,max_salary,title_salary
137,69000000.0,73000000.0,6.9 - 7.3 triệu
138,69000000.0,73000000.0,6.9 - 7.3 triệu
214,15000000.0,2000000.0,1.5 - 2 triệu
265,22000000.0,255000000.0,22 - 25.5 triệu
295,NaN,NaN,Tới 25.5 triệu
299,NaN,NaN,Tới 40.5 triệu
426,85000000.0,15000000.0,8.5 - 15 triệu
550,NaN,NaN,Từ 2.5 triệu
659,1000000.0,15000000.0,1 - 1.5 triệu
863,81000000.0,15000000.0,8.1 - 15 triệu


In [ ]:
# tính average_salary, (min + max) / 2
df['average_salary'] = (df['min_salary'] + df['max_salary']) / 2



In [ ]:
# sort average_salary from min to max
df = df.sort_values(by='average_salary')
df.tail(10)


,title,company_name,title_salary,city_text,exp,item_tag,company_info,job_des,job_require,other_info,company_scale,company_field,position,work_type,education,min_salary,max_salary,average_salary
2329,"Automation Test (Java, Selenium)",Tp&p Technology Co. Ltd.,Tới 35 triệu,Hà Nội,2 năm,Automation Tester,TP&P Technology Co. Ltd.\n \n ...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,100-499,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2331,Data Tester,Tp&p Technology Co. Ltd.,Tới 30 triệu,Hà Nội,2 năm,Manual Tester,TP&P Technology Co. Ltd.\n \n ...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,100-499,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2332,Senior IT BA,Tp&p Technology Co. Ltd.,Tới 40 triệu,Hà Nội,3 năm,Chuyên môn Công nghệ thông tin khác,TP&P Technology Co. Ltd.\n \n ...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,100-499,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2333,DevSecOps,Tp&p Technology Co. Ltd.,Tới 40 triệu,Hà Nội,3 năm,DevOps Engineer,TP&P Technology Co. Ltd.\n \n ...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,100-499,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2337,Chuyên Viên Phát Triển Hệ Thống Giáo Dục Số Ed...,Trung Tâm Dịch Vụ Số Mobifone - Chi Nhánh Tổng...,Tới 20 triệu,Hà Nội,1 năm,Chuyên môn Công nghệ thông tin khác,Trung tâm Dịch vụ số MobiFone - Chi nhánh Tổng...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,100-499,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2338,"Chuyên Viên Lập Trình (Nghỉ Thứ 7 - Chủ Nhật, ...",Trung Tâm Dịch Vụ Số Mobifone - Chi Nhánh Tổng...,Tới 20 triệu,Hà Nội,1 năm,Fullstack Developer,Trung tâm Dịch vụ số MobiFone - Chi nhánh Tổng...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,100-499,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2346,Cộng Tác Viên Thiết Kế,Trường Đại Học Văn Lang,Tới 7 triệu,Hồ Chí Minh,Dưới 1 năm,Thiết kế đồ họa (Graphic Design),Trường Đại học Văn Lang\n \n ...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,1000 - 2999,Giáo dục / Đào tạo,Nhân viên,Thực tập,Đại Học trở lên,NaN,NaN,NaN
2352,Presale Network Engineer,Vdi - Công Ty Cổ Phần Đầu Tư Kỹ Thuật Số Việt,Tới 30 triệu,Hồ Chí Minh,2 năm,Network Engineer,VDI - Công ty Cổ phần Đầu tư Kỹ thuật số Việt\...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,100-499,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2358,IT Communicator,Wanosoft,Tới 35 triệu,Hà Nội,4 năm,IT Comtor,WanoSoft\n \n ...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,10-24,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN
2360,2D Game Artist,Wayfu Studio,Tới 18 triệu,Hà Nội,1 năm,Thiết kế đồ họa (Graphic Design),WAYFU STUDIO\n \n ...,Mô tả công việc\n \n ...,Yêu cầu ứng viên\n \n ...,Thông tin chung\n \n \n \n ...,25-99,IT - Phần mềm,Nhân viên,Toàn thời gian,Đại Học trở lên,NaN,NaN,NaN


In [ ]:
# print 10 dòng đầu tiên của 3 cột mới
print(df[['min_salary', 'max_salary', 'average_salary']].head(10))

      min_salary  max_salary  average_salary
1720   1000000.0   3000000.0       2000000.0
509    1000000.0   3000000.0       2000000.0
2094   2000000.0   3000000.0       2500000.0
819    2000000.0   3000000.0       2500000.0
1383   5000000.0   1000000.0       3000000.0
2281   2000000.0   4000000.0       3000000.0
2123   2000000.0   4000000.0       3000000.0
224    2000000.0   4000000.0       3000000.0
2280   2000000.0   4000000.0       3000000.0
2282   2000000.0   4000000.0       3000000.0


In [ ]:
# if find "Trên" or "Tới" in title_salary, set min_salary = max_salary = average_salary = this number

def process_salary2(text):
    # Loại bỏ khoảng trắng thừa
    text = text.strip()
    
    # Nếu không tìm thấy 'Trên' hoặc 'Tới' thì trả về NaN
    if 'Trên' not in text and 'Tới' not in text:
        return np.nan, np.nan, np.nan
    
    # Loại bỏ các ký tự không phải là số
    number = re.sub(r'\D', '', text)
    
    # Kiểm tra nếu number rỗng
    if not number:
        return np.nan, np.nan, np.nan
    
    # Chuyển đổi thành số nguyên
    number = int(number)
    
    # Nếu tìm thấy 'triệu' hoặc 'tr' thì nhân với 1000000
    if any(unit in text.lower() for unit in ['triệu', 'tr']):
        number *= 1000000
    
    # Nếu tìm thấy 'USD' thì nhân với 25485
    elif 'USD' in text.upper():
        number *= 25485
    
    return number, number, number

# Áp dụng hàm cho các hàng đang bị NaN. nếu không thì bỏ qua
mask = df['min_salary'].isnull()
salary_df = df.loc[mask, 'title_salary'].apply(process_salary2).apply(pd.Series)
salary_df.columns = ['min_salary', 'max_salary', 'average_salary']
df.loc[mask, ['min_salary', 'max_salary', 'average_salary']] = salary_df.values


In [ ]:
# xóa dấu cách thừa trong title_salary
df['title_salary'] = df['title_salary'].apply(lambda x: re.sub(r'\s+', ' ', x))
# if min_salary = Nan, print unique title_salary
print(df[df['min_salary'].isnull()]['title_salary'].unique())



['Từ 1,200 USD' 'Từ 16 triệu' 'Từ 50 triệu' 'Từ 2 triệu' 'Từ 5 triệu'
 'Từ 30 triệu' 'Từ 9 triệu' 'Từ 20 triệu' 'Từ 2.5 triệu' 'Từ 15 triệu'
 'Từ 1 triệu' 'Từ 1.5 triệu' 'Từ 60 triệu' 'Từ 3 triệu' 'Từ 8 triệu'
 'Từ 7 triệu' 'Từ 5.7 triệu' 'Từ 25 triệu' 'Từ 0.0 triệu' 'Từ 1,000 USD'
 'Từ 10 triệu' 'Từ 4 triệu']


In [ ]:
# print min_salary, max_salary, average_salary
print(df[['min_salary', 'max_salary', 'average_salary']].head(100))


      min_salary  max_salary  average_salary
1720   1000000.0   3000000.0       2000000.0
509    1000000.0   3000000.0       2000000.0
2094   2000000.0   3000000.0       2500000.0
819    2000000.0   3000000.0       2500000.0
1383   5000000.0   1000000.0       3000000.0
...          ...         ...             ...
2132   7000000.0  10000000.0       8500000.0
1026   7000000.0  10000000.0       8500000.0
749    7000000.0  10000000.0       8500000.0
2183   7000000.0  10000000.0       8500000.0
2012   8000000.0   9000000.0       8500000.0

[100 rows x 3 columns]


In [45]:
#print null min_salary
print(df['min_salary'].isnull().sum())

42


## Experience

In [ ]:

# duplicate cột exp -> exp_pro
df['exp_pro'] = df['exp']

# chỉ lấy số trong cột exp
def extract_experience(text):
    if pd.isna(text):
        return np.nan
    # Chuyển đổi sang chuỗi để tránh lỗi TypeError
    text = str(text)
    # Tìm kiếm các số trong chuỗi
    numbers = re.findall(r'\d+', text)
    if numbers:
        return int(numbers[0])  # Trả về số đầu tiên tìm thấy
    else:
        return np.nan  # Trả về NaN nếu không tìm thấy số

# Áp dụng hàm cho cột 'exp'
df['exp'] = df['exp'].apply(extract_experience)




[nan  1.  2.  3.  5.  4.]


In [ ]:
# if exp = nan fill = 0
df['exp'] = df['exp'].fillna(0)
print(df['exp'].unique())

[0. 1. 2. 3. 5. 4.]


In [ ]:
#print null experience
print(df['exp'].isnull().sum())

0


# PREPROCESS - MISSING VALUES

### Kiểm tra dữ liệu sau khi thêm cột


In [ ]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 1979 entries, 1720 to 265
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1976 non-null   object 
 1   company_name    1979 non-null   object 
 2   title_salary    1979 non-null   object 
 3   city_text       1979 non-null   object 
 4   exp             1979 non-null   float64
 5   item_tag        1978 non-null   object 
 6   company_info    1979 non-null   object 
 7   job_des         1979 non-null   object 
 8   job_require     1979 non-null   object 
 9   other_info      1979 non-null   object 
 10  company_scale   1979 non-null   object 
 11  company_field   1979 non-null   object 
 12  position        1979 non-null   object 
 13  work_type       1979 non-null   object 
 14  education       1979 non-null   object 
 15  min_salary      1979 non-null   float64
 16  max_salary      1979 non-null   float64
 17  average_salary  1979 non-null   floa

In [ ]:
# Xóa các cột không cần thiết
df = df.drop(columns=['jobdetail', 'jobtitle', 'general_info', 'raw_salary', 'raw_experience', 'Unnamed: 0'])

In [ ]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 2233 entries, 4314 to 3427
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   jobname         2233 non-null   object 
 1   companyscale    2233 non-null   object 
 2   company_field   2233 non-null   object 
 3   company_name    2233 non-null   object 
 4   skill_offer1    2233 non-null   object 
 5   skill_offer2    2233 non-null   object 
 6   skill_offer3    2233 non-null   object 
 7   skill_offer4    2233 non-null   object 
 8   Location        2233 non-null   object 
 9   position        2233 non-null   object 
 10  work_type       2233 non-null   object 
 11  min_salary      2233 non-null   float64
 12  max_salary      2233 non-null   float64
 13  average_salary  2233 non-null   float64
 14  experience      2233 non-null   int64  
dtypes: float64(3), int64(1), object(11)
memory usage: 279.1+ KB
None
         min_salary    max_salary  average_salary   experien

In [ ]:
#Với mỗi cột, đếm xem có bao nhiêu giá trị ''
print((df == '').sum())


jobname             0
companyscale        7
company_field       0
company_name        0
skill_offer1      539
skill_offer2      616
skill_offer3      729
skill_offer4      865
Location           51
position            0
work_type          19
min_salary          0
max_salary          0
average_salary      0
experience          0
dtype: int64


In [ ]:
#delete all row have '' in company_scale, Location, work_type
df = df[df['companyscale'] != '']
df = df[df['Location'] != '']
df = df[df['work_type'] != '']
#detêt all row if min_salary = null
df = df[df['min_salary'].notnull()]

In [ ]:
#print last 5 rows
print(df.tail())

                                    jobname companyscale  company_field  \
3161                            AI Engineer        25-99  It - Phần Mềm   
3652                      Software Engineer        25-99  It - Phần Mềm   
4164  Software Tester (Automation & Manual)        25-99  It - Phần Mềm   
4146                          Manual Tester      100-499  It - Phần Mềm   
3427                    Fullstack Developer      100-499  It - Phần Mềm   

                          company_name       skill_offer1  \
3161  Công Ty Tnhh Công Nghệ Blue Belt   Machine Learning   
3652           Công Ty Tnhh Otani U.p.             Python   
4164           Công Ty Tnhh Otani U.p.             Tester   
4146        Công Ty Cổ Phần Its Global             Tester   
3427        Cty Tnhh R Techno Việt Nam  Phân Tích Yêu Cầu   

              skill_offer2              skill_offer3        skill_offer4  \
3161  Large Language Model             Deep Learning  Tensorflow/pytorch   
3652                 Vuejs  Ch

In [ ]:
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 2175 entries, 4314 to 3427
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   jobname         2175 non-null   object 
 1   companyscale    2175 non-null   object 
 2   company_field   2175 non-null   object 
 3   company_name    2175 non-null   object 
 4   skill_offer1    2175 non-null   object 
 5   skill_offer2    2175 non-null   object 
 6   skill_offer3    2175 non-null   object 
 7   skill_offer4    2175 non-null   object 
 8   Location        2175 non-null   object 
 9   position        2175 non-null   object 
 10  work_type       2175 non-null   object 
 11  min_salary      2175 non-null   float64
 12  max_salary      2175 non-null   float64
 13  average_salary  2175 non-null   float64
 14  experience      2175 non-null   int64  
dtypes: float64(3), int64(1), object(11)
memory usage: 271.9+ KB
None
         min_salary    max_salary  average_salary   experien

# PREPROCESS - TRANSLATE TO ENGLISH

In [ ]:
%pip install googletrans==4.0.0-rc1


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from googletrans import Translator
from tqdm import tqdm

# Các cột cần dịch
columns_to_translate = [
    "jobname",
    "company_field",
    "company_name",
    "skill_offer1",
    "skill_offer2",
    "skill_offer3",
    "skill_offer4",
    "position",
    "work_type"
]

# Khởi tạo đối tượng Translator
translator = Translator()

# Hàm dịch nội dung

def translate_text(text):
    try:
        if pd.notnull(text):  # Kiểm tra nếu ô không rỗng
            translation = translator.translate(text, src='auto', dest='en')
            return translation.text if translation and translation.text else text
        return text
    except Exception as e:
        print(text)
        return text

# Chỉ lấy 5 hàng đầu tiên để thử nghiệm
data_sample = df.head(5).copy()

# Dịch từng cột với tqdm để hiển thị tiến độ
for column in columns_to_translate:
    if column in data_sample.columns:
        print(f"Đang dịch cột: {column}")
        data_sample[column] = [translate_text(text) for text in tqdm(data_sample[column], desc=f"Dịch {column}")]

# Kiểm tra kết quả
print(data_sample)


Đang dịch cột: jobname


Dịch jobname: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


Đang dịch cột: company_field


Dịch company_field: 100%|██████████| 5/5 [00:04<00:00,  1.08it/s]


Đang dịch cột: company_name


...

Đang dịch cột: position


Dịch position: 100%|██████████| 5/5 [00:05<00:00,  1.12s/it]


Đang dịch cột: work_type


Dịch work_type: 100%|██████████| 5/5 [00:05<00:00,  1.11s/it]

                                    jobname companyscale  company_field  \
4314                      Software business        25-99        Finance   
319                     Fullstack Developer     500-1000  IT - Software   
332                          Game Developer        25-99  IT - Software   
1233          Product Owner/Product Manager        25-99  IT - Software   
3775  Software Tester (Automation & Manual)        25-99  IT - Software   

                                   company_name        skill_offer1  \
4314                Paytech Joint Stock Company             Present   
319   CMC Technology & Solution Company Limited                       
332                              Gamify Studios                       
1233                             Gamify Studios               Agile   
3775         Live Group Vietnam Company Limited  Project management   

                skill_offer2      skill_offer3           skill_offer4  \
4314  Building relationships         Negotiate   

## Translate Jobname

In [ ]:
# Delete all in ( ) in jobname
df['jobname'] = df['jobname'].apply(lambda x: re.sub(r'\(.*\)', '', x).strip())

In [ ]:
# save jobname to temp array
jobname_array = df['jobname'].unique()
jobname_array

array(['Kinh doanh phần mềm', 'Fullstack Developer', 'Game Developer',
       'Product Owner/Product Manager', 'Software Tester', 'UI/UX Design',
       'System Architect', 'Business Analyst', 'Công nghệ thông tin khác',
       'Data Analyst', 'Thiết kế đồ họa', 'Mobile Developer',
       'Frontend Developer', 'Bán hàng kỹ thuật IT', 'IT Consultant',
       'Game Tester', 'Backend Developer', 'AI Engineer',
       'Software Engineer', 'Automation Tester', 'Kỹ sư cầu nối BrSE',
       'Sales IT Phần mềm khác', 'Database Administrator',
       'Brand Marketing', 'DevOps Engineer',
       'Kinh doanh Domain/Hosting/Server', 'System Engineer',
       'Manual Tester', 'Data Scientist', 'Illustration', 'Game Design',
       'Concept Artist', 'Embedded Engineer/Lập trình nhúng', 'Kỹ sư IoT',
       'IT Helpdesk/IT support', 'Data Labeling', 'Kỹ thuật IT',
       'Data Engineer', 'Cloud Engineer', 'Vị trí Game Development khác',
       'Animation Design', 'Quản trị và vận hành bảo mật',
      

In [ ]:
# translate jobname_array
jobname_array = [translate_text(text) for text in tqdm(jobname_array, desc="Dịch jobname")]


Dịch jobname: 100%|██████████| 73/73 [00:48<00:00,  1.51it/s]


In [ ]:
# create dictionary to save jobname_array
jobname_dict = dict(zip(df['jobname'].unique(), jobname_array))
# replace jobname by jobname_dict
df['jobname'] = df['jobname'].replace(jobname_dict)

In [ ]:
# print unique jobname
print(df['jobname'].unique())

['Software business' 'Fullstack Developer' 'Game Developer'
 'Product Owner/Product Manager' 'Software tester' 'UI/UX Design'
 'System Architect' 'Business Analyst' 'Other information technology'
 'Data Analyst' 'Graphic design' 'Mobile Developer' 'Frontend Developer'
 'IT technical sales' 'IT Consultant' 'Game Tester' 'Backend Developer'
 'AI Engineer' 'Software Engineer' 'Automation Tester'
 'Bridge Bridge Bridge Bridge' 'Sales IT other software'
 'Database Administrator' 'Brand Marketing' 'DevOps Engineer'
 'Domain/hosting/server business' 'System Engineer' 'Manual Tester'
 'Data Scientist' 'Illustration' 'Game Design' 'Concept Artist'
 'Embedded Engineer/Dip programming' 'IoT engineer'
 'IT Helpdesk/IT support' 'Data Labeling' 'IT technique' 'Data Engineer'
 'Cloud Engineer' 'Other Development game position' 'Animation Design'
 'Management and operation of confidentiality' 'System Administrator'
 'QA Engineer' 'Other subject teachers' 'Computer teacher'
 'Network Engineer' 'Process

## Translate company_field

In [ ]:
# save company_field to temp array
company_field_array = df['company_field'].unique()
company_field_array


array(['Tài Chính', 'It - Phần Mềm', 'Viễn Thông', 'Ngân Hàng',
       'Bán Lẻ - Hàng Tiêu Dùng - Fmcg', 'Bất Động Sản',
       'Internet / Online', 'Khác', 'Tư Vấn', 'Xuất Nhập Khẩu',
       'Xây Dựng', 'Điện Tử / Điện Lạnh',
       'Marketing / Truyền Thông / Quảng Cáo', 'Du Lịch',
       'Giáo Dục / Đào Tạo', 'Thương Mại Điện Tử',
       'Dược Phẩm / Y Tế / Công Nghệ Sinh Học',
       'Agency (marketing/advertising)', 'Tự Động Hóa', 'Sản Xuất',
       'Giải Trí', 'Nhân Sự', 'Nhà Hàng / Khách Sạn', 'In Ấn / Xuất Bản',
       'It - Phần Cứng', 'Logistics - Vận Tải', 'Thời Trang',
       'Chứng Khoán', 'Thiết Kế / Kiến Trúc', 'Kế Toán / Kiểm Toán',
       'Nông Lâm Ngư Nghiệp', 'Bảo Trì / Sửa Chữa', 'Bảo Hiểm', 'Cơ Khí',
       'Năng Lượng'], dtype=object)

In [ ]:
# translate company_field_array
company_field_array = [translate_text(text) for text in tqdm(company_field_array, desc="Dịch company_field")]

# create dictionary to save company_field_array
company_field_dict = dict(zip(df['company_field'].unique(), company_field_array))
# replace company_field by company_field_dict
df['company_field'] = df['company_field'].replace(company_field_dict)

Dịch company_field: 100%|██████████| 35/35 [00:27<00:00,  1.27it/s]


In [ ]:
df['company_field'].unique()

array(['Finance', 'IT - Software', 'Telecommunications', 'Bank',
       'Retail - Consumer goods - FMCG', 'Real estate',
       'Internet / Online', 'Other', 'Advise', 'Import and export',
       'Build', 'Electronics / Refrigeration',
       'Marketing / Communications / Advertising', 'Tourism',
       'Education / Training', 'E -commerce',
       'Pharmaceutical / Health / Biotechnology',
       'Agency (marketing/advertising)', 'Automation', 'Manufacture',
       'Entertainment', 'Personnel', 'Restaurant / hotel',
       'Print / publish', 'IT - Hardware', 'Logistics - Transport',
       'Fashion', 'Stock', 'Design / Architecture',
       'Accounting / Auditing', 'Agriculture and forestry',
       'Maintenance / Repair', 'Insurance', 'Mechanical', 'Energy'],
      dtype=object)

## Translate position

In [ ]:
# save position to temp array
position_array = df['position'].unique()
position_array

array(['Trưởng/Phó phòng', 'Nhân viên', 'Quản lý / Giám sát',
       'Thực tập sinh', 'Trưởng nhóm', 'Trưởng chi nhánh', 'Giám đốc',
       'Phó giám đốc'], dtype=object)

In [ ]:
#translate position_array
position_array = [translate_text(text) for text in tqdm(position_array, desc="Dịch position")]

# create dictionary to save position_array
position_dict = dict(zip(df['position'].unique(), position_array))
# replace position by position_dict
df['position'] = df['position'].replace(position_dict)


Dịch position: 100%|██████████| 8/8 [00:05<00:00,  1.36it/s]


In [ ]:
df.head(10)

,jobname,companyscale,company_field,company_name,skill_offer1,skill_offer2,skill_offer3,skill_offer4,Location,position,work_type,min_salary,max_salary,average_salary,experience
4314,Software business,25-99,Finance,Công Ty Cổ Phần Paytech,Thuyết Trình,Xây Dựng Mối Quan Hệ,Đàm Phán,Quản Lý Dự Án,Hà Nội,Head/Deputy Head,Toàn thời gian,0.0,0.0,0.0,4
319,Fullstack Developer,500-1000,IT - Software,Công Ty Tnhh Tổng Công Ty Công Nghệ & Giải Phá...,,,,,Hà Nội,Staff,Toàn thời gian,0.0,0.0,0.0,4
332,Game Developer,25-99,IT - Software,Gamify Studios,,,,,Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,4
1233,Product Owner/Product Manager,25-99,IT - Software,Gamify Studios,Agile,Scrum,Product Strategy,Product Roadmap,Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,1
3775,Software tester,25-99,IT - Software,Công Ty Tnhh Live Group Việt Nam,Quản Lý Dự Án,Kiểm Thử Phần Mềm,Lập Kế Hoạch Kiểm Thử,Thiết Kế Test Case,Hồ Chí Minh,Management / supervision,Toàn thời gian,0.0,0.0,0.0,4
4270,UI/UX Design,25-99,IT - Software,Gamify Studios,Thiết Kế Giao Diện Người Dùng (ui),Thiết Kế Trải Nghiệm Người Dùng (ux),Nghiên Cứu Người Dùng,"Sử Dụng Phần Mềm Thiết Kế (figma, Adobe Xd, Sk...",Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,3
3808,System Architect,5000 - 9999,Telecommunications,Công Ty Cổ Phần Viễn Thông Fpt,Quản Lý Dự Án,Thiết Kế Hệ Thống,Lập Trình Hướng Đối Tượng,Phát Triển Phần Mềm,Hồ Chí Minh,Staff,Toàn thời gian,0.0,0.0,0.0,5
2867,Business Analyst,25-99,IT - Software,Công Ty Tnhh Công Nghệ Blue Belt,Kỹ Năng Lãnh Đạo,Quản Lý Dự Án,Phân Tích Nghiệp Vụ,Giao Tiếp Hiệu Quả,Hà Nội,Head/Deputy Head,Toàn thời gian,0.0,0.0,0.0,5
3643,Fullstack Developer,100-499,IT - Software,Opencommerce Group,Python,Vuejs,Product Development,Sql/nosql,Hà Nội,Staff,Toàn thời gian,0.0,0.0,0.0,4
884,Other information technology,5000 - 9999,Bank,Ngân Hàng Tmcp An Bình Pro Company,,,,,Hà Nội,Staff,Toàn thời gian,0.0,0.0,0.0,3


## Translate work_type

In [ ]:
# save work_type to temp array
work_type_array = df['work_type'].unique()
work_type_array


array(['Toàn thời gian', 'Thực tập', 'Bán thời gian'], dtype=object)

In [ ]:
#translate work_type_array
work_type_array = [translate_text(text) for text in tqdm(work_type_array, desc="Dịch work_type")]

# create dictionary to save work_type_array
work_type_dict = dict(zip(df['work_type'].unique(), work_type_array))
# replace work_type by work_type_dict
df['work_type'] = df['work_type'].replace(work_type_dict)


Dịch work_type: 100%|██████████| 3/3 [00:02<00:00,  1.21it/s]


## Skill offer

In [ ]:
import json

# Đọc file JSON mới được tải lên
json_path = 'final_skill_groups.json'
with open(json_path, 'r', encoding='utf-8') as f:
    skill_groups = json.load(f)

# Tạo cột mới trong DataFrame để chứa nhóm kỹ năng
def classify_skill(skill):
    if pd.isna(skill) or skill == '':
        return ''
    for group, skills in skill_groups.items():
        if skill in skills:
            return group
    return 'Other'

# Áp dụng phân loại cho từng cột kỹ năng
for col in df.columns:
    df[f'{col}_group'] = df[col].apply(lambda x: classify_skill(x))

# Loại bỏ giá trị trùng lặp trong cùng một hàng
group_cols = [col for col in df.columns if '_group' in col]

# Xử lý từng hàng để loại bỏ giá trị trùng lặp
for index, row in df.iterrows():
    seen = set()
    for col in group_cols:
        if row[col] in seen:
            df.at[index, col] = ''  # Xóa giá trị trùng lặp
        else:
            seen.add(row[col])
 
# Xuất file CSV sau khi ghép nhóm kỹ năng
output_csv_path = 'cleanded_database.csv'
df.to_csv(output_csv_path, index=False)

output_csv_path

'cleanded_database.csv'